In [2]:
import langchain_community
import langchain_text_splitters
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter
import os
import pprint

In [114]:
# Load several documents

storage = []

for root, dirs, files in os.walk("/data/local/aparraga/Bioinformatician/RAG/Publications",):
    for file in files:


        loader = PyPDFLoader(os.path.join(root,file))
        pages = loader.load_and_split()

        # split it into chunks
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
        docs = text_splitter.split_documents(pages)
        storage.extend(docs) # This  is a list of lists


Multiple definitions in dictionary at byte 0x2325b for key /MediaBox
Multiple definitions in dictionary at byte 0x2348a for key /MediaBox
Multiple definitions in dictionary at byte 0x23604 for key /MediaBox
Multiple definitions in dictionary at byte 0x237a6 for key /MediaBox
Multiple definitions in dictionary at byte 0x239b8 for key /MediaBox
Multiple definitions in dictionary at byte 0x23b3a for key /MediaBox
Multiple definitions in dictionary at byte 0x23ca4 for key /MediaBox
Multiple definitions in dictionary at byte 0x23e1e for key /MediaBox
Multiple definitions in dictionary at byte 0x23f90 for key /MediaBox
Multiple definitions in dictionary at byte 0x2421a for key /MediaBox
Multiple definitions in dictionary at byte 0x2445c for key /MediaBox


In [139]:
print(len(storage))
storage[66].metadata

1313


{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2024-06-22T13:40:06+08:00',
 'author': 'Bruce A. Lessey, Allan Dong, Jeffrey L. Deaton, Daniel Angress, Ricardo F. Savaris and Stephen J. Walker',
 'keywords': 'endometriosis; implantation; infertility; recurrent pregnancy loss; inflammation',
 'moddate': '2024-06-22T07:43:33+02:00',
 'subject': 'Unexplained euploid embryo transfer failure (UEETF) is a frustrating and unanswered conundrum accounting for 30 to 50% of failures in in vitro fertilization using preimplantation genetic testing for aneuploidy (PGT-A). Endometriosis is thought by many to account for most of such losses and menstrual suppression or surgery prior to the next transfer has been reported to be beneficial. In this study, we performed endometrial biopsy in a subset of women with UEETF, testing for the oncogene BCL6 and the histone deacetylase SIRT1. We compared 205 PGT-A cycles outcomes and provide those results following treatment wi

In [4]:
# load the document and split it into pages
loader = PyPDFLoader("/data/local/aparraga/Bioinformatician/RAG/Publications/Parraga-Leo2023.pdf")
pages = loader.load_and_split()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
storage = text_splitter.split_documents(pages)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipykernel_2455317/1728583740.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/home/aparraga/anaconda3/envs/RAG/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# create the chroma client
import uuid
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(host='localhost', port=8000, settings=Settings(allow_reset=True))
# client.reset()  # resets the database
# collection = client.create_collection("tfm_APL")
# collection = client.get_collection('tfm')

# for doc in storage:
#     collection.add(
#         ids=[str(uuid.uuid1())], metadatas=doc.metadata, documents=doc.page_content
#     )

# tell LangChain to use our client and collection name
db = Chroma(
    client=client,
    collection_name="tfm",
    embedding_function=embedding_function,
)

/tmp/ipykernel_2455317/2599506003.py:17: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [7]:
query = "What is the implantation failure and what is its cause? Give me a detail explanation of implantation failure, not only about RIF"
docs = db.similarity_search(query)

fullcontent =''

for doc in docs:
    fullcontent ='. '.join([fullcontent,doc.page_content])
#print(docs[0].page_content)
#print(len(docs))

print(fullcontent)

. 1. Introduction
Recurrent implantation failure (RIF) is a highly demanding and complex clinical
phenomenon that, despite advancements in assisted reproductive technology, remains a
major issue affecting approximately 15% of women undergoing in vitro fertilization [1].
Despite the extensive relevant literature, there is still no widely accepted definition. The
definition that until recently had the broadest acceptance for RIF is that of Coughlan et al.,
who define RIF as the failure to achieve clinical pregnancy after the transfer of at least four
J. Clin. Med.2024, 13, 2145. https://doi.org/10.3390/jcm13072145 https://www.mdpi.com/journal/jcm. 1. Introduction
Recurrent implantation failure (RIF) is a highly demanding and complex clinical
phenomenon that, despite advancements in assisted reproductive technology, remains a
major issue affecting approximately 15% of women undergoing in vitro fertilization [1].
Despite the extensive relevant literature, there is still no widely accepted 

In [122]:
uuid.uuid1()
pprint.pprint(storage[66].metadata)

{'author': 'Bruce A. Lessey, Allan Dong, Jeffrey L. Deaton, Daniel Angress, '
           'Ricardo F. Savaris and Stephen J. Walker',
 'creationdate': '2024-06-22T13:40:06+08:00',
 'creator': 'LaTeX with hyperref',
 'keywords': 'endometriosis; implantation; infertility; recurrent pregnancy '
             'loss; inflammation',
 'moddate': '2024-06-22T07:43:33+02:00',
 'page': 3,
 'page_label': '4',
 'producer': 'pdfTeX-1.40.25',
 'source': '/data/local/aparraga/Bioinformatician/RAG/Publications/38999962.pdf',
 'subject': 'Unexplained euploid embryo transfer failure (UEETF) is a '
            'frustrating and unanswered conundrum accounting for 30 to 50% of '
            'failures in in vitro fertilization using preimplantation genetic '
            'testing for aneuploidy (PGT-A). Endometriosis is thought by many '
            'to account for most of such losses and menstrual suppression or '
            'surgery prior to the next transfer has been reported to be '
            'beneficia

In [157]:
def get_system_message_rag(content):
        return f"""You are an expert consultant helping executive advisors to get relevant information from scientific articles and code related to reproduction and bioinformatics.

        Generate your response by following the steps below:
        1. Recursively break down the question into smaller questions to better understand it.
        2. For each question/directive:
            2a. Select the most relevant information from the context in light of the conversation history.
        3. Generate a draft response using selected information.
        4. Remove duplicate content from draft response.
        5. Generate your final response after adjusting it to increase accuracy and relevance.
        6. Do not try to summarize the answers, explain it properly.
        6. Only show your final response! 
        
        Constraints:
        1. DO NOT PROVIDE ANY EXPLANATION OR DETAILS OR MENTION THAT YOU WERE GIVEN CONTEXT. Only do that when questions are related to coding.
        2. Don't mention that you are not able to find the answer in the provided context.
        3. Ignore the part of the content that only contains references.
        3. Don't make up the answers by yourself.
        4. Try your best to provide answer from the given context.

        CONTENT:
        {content}
        """

In [132]:
def get_ques_response_prompt(question):
    return f"""
    ==============================================================
    Based on the above context, please provide the answer to the following question:
    {question}
    """

In [158]:
from ollama import Client
client = Client(host='http://localhost:11434')
stream = client.chat(model='gemma3:12b', messages=[
{"role": "system", "content": get_system_message_rag(fullcontent)},            
{"role": "user", "content": get_ques_response_prompt(query)}
],stream=True)

In [ ]:

# stream = client.chat(model='mistral', messages=[
#     {"role": "system", "content": get_system_message_rag(content)},            
#     {"role": "user", "content": get_ques_response_prompt(query)}
#     ],stream=False)

In [159]:

print(get_system_message_rag(fullcontent))
print(get_ques_response_prompt(query))
print("####### THINKING OF ANSWER............ ")
full_answer = ''

for chunk in stream:
    # print(chunk['message']['content'], end='', flush=True)
    full_answer =''.join([full_answer,chunk['message']['content']])

You are an expert consultant helping executive advisors to get relevant information from scientific articles and code related to reproduction and bioinformatics.

        Generate your response by following the steps below:
        1. Recursively break down the question into smaller questions to better understand it.
        2. For each question/directive:
            2a. Select the most relevant information from the context in light of the conversation history.
        3. Generate a draft response using selected information.
        4. Remove duplicate content from draft response.
        5. Generate your final response after adjusting it to increase accuracy and relevance.
        6. Do not try to summarize the answers, explain it properly.
        6. Only show your final response! 
        
        Constraints:
        1. DO NOT PROVIDE ANY EXPLANATION OR DETAILS OR MENTION THAT YOU WERE GIVEN CONTEXT. Only do that when questions are related to coding.
        2. Don't mention that 

In [161]:
pprint.pprint(full_answer)

('Implantation failure refers to the inability of an embryo to successfully '
 'attach to and be supported by the uterine lining (endometrium), preventing '
 'pregnancy from progressing. This can occur in the context of Recurrent '
 "Implantation Failure (RIF), but it's a broader phenomenon that can also "
 'contribute to recurrent miscarriage.\n'
 '\n'
 '**RIF Definition:** According to the European Society of Human Reproduction '
 'and Embryology (ESHRE) guidelines, RIF is defined as the inability to '
 'achieve a clinical pregnancy after the transfer of at least five '
 'high-quality embryos in at least two or more IVF cycles. This suggests that '
 "the embryo doesn't implant properly.\n"
 '\n'
 "**Distinction from Recurrent Miscarriage:** It's important to differentiate "
 'implantation failure from recurrent miscarriage. In recurrent miscarriage, '
 'the embryo initially implants and begins to develop, but is subsequently '
 'lost. These conditions require different diagnostic and

In [112]:
help(client.chat)

Help on method chat in module ollama._client:

chat(model: str = '', messages: Optional[Sequence[Union[Mapping[str, Any], ollama._types.Message]]] = None, *, tools: Optional[Sequence[Union[Mapping[str, Any], ollama._types.Tool, Callable]]] = None, stream: bool = False, think: Optional[bool] = None, format: Union[Literal['', 'json'], dict[str, Any], NoneType] = None, options: Union[Mapping[str, Any], ollama._types.Options, NoneType] = None, keep_alive: Union[float, str, NoneType] = None) -> Union[ollama._types.ChatResponse, collections.abc.Iterator[ollama._types.ChatResponse]] method of ollama._client.Client instance
    Create a chat response using the requested model.
    
    Args:
      tools:
        A JSON schema as a dict, an Ollama Tool or a Python Function.
        Python functions need to follow Google style docstrings to be converted to an Ollama Tool.
        For more information, see: https://google.github.io/styleguide/pyguide.html#38-comments-and-docstrings
      stream: 